# Identification of Potential Drug Targets of Inflammatory Bowel Disease

*Please note: This notebook uses open access data*  

#### Fan Wang
#### July 31 2022

Systematic MR of molecular phenotypes such as proteins and expression of transcript levels offer enormous potential to prioritise drug targets for further investigation. However, many genes and gene products are not easily druggable, so some potentially important causal genes may not offer an obvious route to intervention.

A parallel problem is that current GWAS of molecular phenotypes have limited sample sizes and limited protein coverages. A potential way to address both these problems is to use protein-protein interaction (PPI) information to identify druggable targets which are linked to a non-druggable, but robustly causal target. Their relationship to the causal target increases our confidence in their potential causal role even if the initial evidence of effect is below our multiple-testing threshold.

This notebook demonstrates an approach to query data in [EpiGraphDB](https://api.epigraphdb.org/) to prioritize potential alternative drug targets in the same PPI network for Inflammatory Bowel Disease (IBD), as follows:

* For an existing drug target of interests, we use PPI networks to search for its directly interacting genes that are evidenced to be druggable.
* We then examine the causal evidence of these candidate genes on the disease.
* We also examine the literature evidence of these candidate genes on the disease.
* Then we query the metadata including meta nodes and meta edges and the overall schema.

In [ ]:
import json
from pprint import pprint
from pprint import pformat
import matplotlib
import matplotlib.pyplot as plt

get_ipython().run_line_magic("config", "InlineBackend.figure_format = 'svg'")
import networkx as nx
import requests
import pandas as pd

First, we will ping the API to check our connection. Here we use the .get() method to send a GET request to the `/ping` endpoint of the API.

In [ ]:
API_URL = "https://api.epigraphdb.org"
endpoint = "/ping"
response_object = requests.get(API_URL + endpoint)
GENE_NAME = "IL23R"
OUTCOME_TRAIT = "Inflammatory bowel disease"

# Check that the ping was sucessful
try:
    response_object.raise_for_status()
    print("If this line gets printed, ping was sucessful.")
except requests.exceptions.HTTPError as err:
    print(err)

### 1. Using PPI networks for alternative drug targets search
The assumption here is that the most likely alternative targets are either directly interacting with IL23R or somewhere in the PPI network. In this example, we consider only genes that were found to interact with *IL23R* via direct protein-protein interactions, and require that those interacting proteins should also be druggable.

The thousands of genes are classified with regard to their druggability by Finan et al. 2017, where the Tier 1 category refers to approved drugs or those in clinical testing while for other tier categories the druggability confidence drops in order Tier 2 and then Tier 3.

In [ ]:
def get_drug_targets_ppi(gene_name):
    endpoint = "/gene/druggability/ppi"
    url = f"{API_URL}{endpoint}"
    params = {"gene_name": gene_name}
    r = requests.get(url, params=params)
    r.raise_for_status()
    df = pd.json_normalize(r.json()["results"])
    return df


GENE_NAME = "IL23R"
OUTCOME_TRAIT = "Inflammatory bowel disease"
ppi_df = get_drug_targets_ppi(gene_name=GENE_NAME)
ppi_df

For further analysis we select the gene of interest (*IL23R*) as well as its interacting genes with Tier 1 druggability.

In [ ]:
def get_gene_list(ppi_df, include_primary_gene: bool = True):
    if include_primary_gene:
        gene_list = list(ppi_df["g1.name"].drop_duplicates()) + list(
            ppi_df.query("`g2.druggability_tier` == 'Tier 1'")["g2.name"]
        )
    else:
        gene_list = list(ppi_df.query("`g2.druggability_tier` == 'Tier 1'")["g2.name"])
    return gene_list


gene_list = get_gene_list(ppi_df)
gene_list

### 2. Looking for literature evidence
EpiGraphDB facilitates fast processing of this information by allowing access to a host of literature-mined relationships that have been structured into semantic triples. These take the general form (subject, predicate, object) and have been generated using contemporary natural language processing techniques applied to a massive amount of published biomedical research papers by [SemMedDB](https://lhncbc.nlm.nih.gov/ii/tools/SemRep_SemMedDB_SKR.html). In the following section we will query the API for the literature relationship between a given gene, *IL23R* (several studies confirmed IL23R associations in independent cohorts of patients with Crohn's disease or ulcerative colitis) and an outcome trait, Inflammatory bowel disease.

In [ ]:
def extract_literature(outcome_trait, gene_list):
    def per_gene(gene_name):
        endpoint = "/gene/literature"
        url = f"{API_URL}{endpoint}"
        params = {"gene_name": gene_name, "object_name": outcome_trait.lower()}
        r = requests.get(url, params=params)
        try:
            r.raise_for_status()
            res_df = pd.json_normalize(r.json()["results"])
            if len(res_df) > 0:
                res_df = res_df.assign(
                    literature_count=lambda df: df["pubmed_id"].apply(lambda x: len(x))
                )
            return res_df
        except:
            return None

    res_df = pd.concat(
        [per_gene(gene_name=gene_name) for gene_name in gene_list]
    ).reset_index(drop=True)
    return res_df


literature_df = extract_literature(outcome_trait=OUTCOME_TRAIT, gene_list=gene_list)
literature_df

### 3. Using Mendelian randomization results for causal effect estimation
The next step is to find out whether any of these genes have a comparable and statistically plausable effect on IBD.

In [ ]:
def extract_mr(outcome_trait, gene_list, qtl_type):
    endpoint = "/xqtl/single-snp-mr"
    url = f"{API_URL}{endpoint}"

    def per_gene(gene_name):
        params = {
            "exposure_gene": gene_name,
            "outcome_trait": outcome_trait,
            "qtl_type": qtl_type,
            "pval_threshold": 1e-5,
        }
        r = requests.get(url, params=params)
        try:
            r.raise_for_status()
            df = pd.json_normalize(r.json()["results"])
            return df
        except:
            return None

    res_df = pd.concat(
        [per_gene(gene_name=gene_name) for gene_name in gene_list]
    ).reset_index(drop=True)
    return res_df


# Search for both pqtl and eqtl
xqtl_df = pd.concat(
    [
        extract_mr(
            outcome_trait=OUTCOME_TRAIT, gene_list=gene_list, qtl_type=qtl_type
        ).assign(qtl_type=qtl_type)
        for qtl_type in ["pQTL", "eQTL"]
    ]
).reset_index(drop=True)
xqtl_df

### 4. Query metadata

Here we query for the metadata information using the endpoint `GET /meta/schema`, which will be used for downstream processing.

In [ ]:
endpoint = "/meta/schema"
params = {"graphviz": False, "plot": False}
r = requests.get(f"{API_URL}{endpoint}", params=params)
r.raise_for_status()
metadata = r.json()

# Preview of metadata information
keys = metadata.keys()
print(pformat(keys), "\n")
for key in list(keys):
    print(f"# {key}:")
    print(pformat(metadata[key])[:1000], "\n")

We can extract the specific meta node information as a pandas dataframe from the metadata.

In [ ]:
meta_node_df = pd.DataFrame.from_dict(metadata["nodes"], orient="index")

(
    meta_node_df.sort_index().assign(
        count=lambda df: df["count"].apply(lambda x: f"{x:,}")
    )
)

We can also extract the meta relationship (edge) information, and the connections.

In [ ]:
meta_rel_df = pd.DataFrame.from_dict(metadata["edges"], orient="index").merge(
    pd.DataFrame.from_dict(
        {_["rel"]: _ for _ in metadata["connections"]}, orient="index"
    )[["from_node", "to_node"]],
    left_index=True,
    right_index=True,
)

(
    meta_rel_df.sort_values(by=["from_node", "to_node"]).assign(
        count=lambda df: df["count"].apply(lambda x: f"{x:,}")
    )
)

We can generate a network diagram of the graph db schema using `networkx`.

In [ ]:
graph = nx.from_pandas_edgelist(meta_rel_df, source="from_node", target="to_node")
cmap = matplotlib.colors.ListedColormap(["dodgerblue", "lightgray", "darkorange"])
meta_rel_df["from_node"] = pd.Categorical(meta_rel_df["from_node"])
f = plt.figure(figsize=(10, 10))
f.tight_layout()
plt.subplot(1, 1, 1)
nx.draw(
    G=graph,
    with_labels=True,
    node_size=3000,
    edgecolors="gray",
    node_color="skyblue",
    font_size=10,
    font_weight="bold",
    width=0.75,
)

### Reference

* Duerr RH, Taylor KD, Brant SR, Rioux JD, Silverberg MS, Daly MJ, Steinhart AH, Abraham C, Regueiro M, Griffiths A, others. 2006. A genome-wide association study identifies IL23R as an inflammatory bowel disease gene. Science 314:1461–1463.

* Finan C, Gaulton A, Kruger FA, Lumbers RT, Shah T, Engmann J, Galver L, Kelley R, Karlsson A, Santos R, others. 2017. The druggable genome and support for target identification and validation in drug development. Science translational medicine 9:eaag1166.

* Momozawa Y, Mni M, Nakamura K, Coppieters W, Almer S, Amininejad L, Cleynen I, Colombel J-F, De Rijk P, Dewit O, others. 2011. Resequencing of positional candidates identifies low frequency IL23R coding variants protecting against inflammatory bowel disease. Nature genetics 43:43–47.

* Zheng J, Brumpton BM, Bronson PG, Liu Y, Haycock P, Elsworth B, Haberland V, Baird D, Walker V, Robinson JW, John S, Prins B, Runz H, Nelson MR, Hurle M, Hemani G, Asvold BO, Butterworth A, Smith GD, Scott RA, Gaunt TR. 2019. Systematic Mendelian randomization and colocalization analyses of the plasma proteome and blood transcriptome to prioritize drug targets for complex disease.